In [5]:
# Data collecting and wrangling for IMDB project

In [2]:
!pip install imdbpie

  Running setup.py bdist_wheel for cachecontrol ... - done
  Stored in directory: /Users/tianjiaoxie/Library/Caches/pip/wheels/9b/94/d2/1793b004461b5bc238a89e260cd2b9f770437c42424fdd0943
Successfully built cachecontrol
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import os
import subprocess
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
import scipy
import requests
from imdbpie import Imdb
import nltk
import matplotlib.pyplot as plt
import urllib2
from bs4 import BeautifulSoup
import nltk
import collections
import re
import csv

%matplotlib inline

//anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [4]:
imdb = Imdb()
imdb = Imdb(anonymize=True)

#imdb.top_250()

In [4]:
movies = []
for dct in imdb.top_250():
    del dct['image']
    movies.append(dct)

In [6]:
Can_rate = []
Vote = []
Rate = []
ID = []
Year = []
Title = []
Type = []
for dct in movies:
    Vote.append(dct['num_votes'])
    Rate.append(dct['rating'])
    ID.append(dct['tconst'])
    Year.append(dct['year'])
    Title.append(dct['title'])
    Can_rate.append(dct['can_rate'])
    Type.append(dct['type'])

In [7]:
df = pd.DataFrame({
        "ID": ID,
        'Title': Title,
        'Year': Year,
        'Vote': Vote,
        'Rate': Rate,
        "Can_rate": Can_rate,
        'Type': Type
    })

# Delete 'Can_rate' and 'Type' column coz all rows are same
df.drop('Can_rate',axis=1,inplace=True)
df.drop('Type',axis=1,inplace=True)

#### To save Top_250 movies into csv
df.to_csv('top_movies.csv',encoding='utf-8')

In [66]:
id_list = list(df.ID)

comments = [[] for i in range(250)]
author_score = [[] for i in range(250)]

In [72]:
for i in id_list:
    url = 'http://www.imdb.com/title/' + i + '/' + 'reviews?start=0' 
    contents = BeautifulSoup(urllib2.urlopen(url),"lxml")
    content = contents.find('div',attrs={'class':'reviews'})
    text = contents.find('td',attrs={'align':'right'}).getText()
    n = int(text[1:text.index('r')].strip()) + 10
    for p in range(0,n,10):
        # comments
        url = 'http://www.imdb.com/title/' + i + '/' + 'reviews?start=' + str(p) 
        contents = BeautifulSoup(urllib2.urlopen(url),"lxml")
        content = contents.find('div',attrs={'id':'tn15content'})
        review = content.find_all('p')
        for rew in review:
            comments[id_list.index(i)].append(rew.text) 
        # uid and stars
        
        for user in content.find_all('div'):
            for a in user.find_all('a',href=True)[:1]:
                author_score[id_list.index(i)].append(a.get("href"))
                for img in user.find_all('img', alt=True):
                    author_score[id_list.index(i)].append(img.get('alt'))


    print '# '+str(id_list.index(i)+1) + ' finished'


# 1 finished
# 2 finished
# 3 finished
# 4 finished
# 5 finished
# 6 finished
# 7 finished
# 8 finished
# 9 finished
# 10 finished
# 11 finished
# 12 finished
# 13 finished
# 14 finished
# 15 finished
# 16 finished
# 17 finished
# 18 finished
# 19 finished
# 20 finished
# 21 finished
# 22 finished
# 23 finished
# 24 finished
# 25 finished
# 26 finished
# 27 finished
# 28 finished
# 29 finished
# 30 finished
# 31 finished
# 32 finished
# 33 finished
# 34 finished
# 35 finished
# 36 finished
# 37 finished
# 38 finished
# 39 finished
# 40 finished
# 41 finished
# 42 finished
# 43 finished
# 44 finished
# 45 finished
# 46 finished
# 47 finished
# 48 finished
# 49 finished
# 50 finished
# 51 finished
# 52 finished
# 53 finished
# 54 finished
# 55 finished
# 56 finished
# 57 finished
# 58 finished
# 59 finished
# 60 finished
# 61 finished
# 62 finished
# 63 finished
# 64 finished
# 65 finished
# 66 finished
# 67 finished
# 68 finished
# 69 finished
# 70 finished
# 71 finished
# 72 finished
#

error: [Errno 54] Connection reset by peer

In [9]:
###### To delete "*** This review may contain spoilers ***" and 'Add another review' in comments
reviews = [[] for i in range(250)]

for comment in comments:
    for s in comment:
        if s != "*** This review may contain spoilers ***" and s != 'Add another review':
            reviews[comments.index(comment)].append(s)
        

In [15]:
##### To split author_score into uid and star 
uid = [[] for i in range(250)]
star = [[] for i in range(250)]

for uss in author_score:
    for i in uss:
        try:
            if i.startswith('/user/') and uss[uss.index(i)+1].startswith('/user/'):
                uid[author_score.index(uss)].append(i)
                star[author_score.index(uss)].append('None')
            elif (i.startswith('/user/')) and uss[uss.index(i)+1].endswith('/10'):
                uid[author_score.index(uss)].append(i)
                star[author_score.index(uss)].append(uss[uss.index(i)+1])
        except IndexError: # catch the error
            uid[author_score.index(uss)].append('error')
            star[author_score.index(uss)].append('error')
            continue

In [19]:
import pickle
with open('reviews','wb') as f:
    pickle.dump(reviews,f)

import pickle
with open('uid','wb') as f:
    pickle.dump(uid,f)

import pickle
with open('star','wb') as f:
    pickle.dump(star,f)

### To transfer data into Pandas DataFrame

In [2]:
import pickle
with open('../../../IMDB_250_Backup/Useless/star','rb') as f:
    star = pickle.load(f)

In [3]:
import pickle
with open('../../../IMDB_250_Backup/Useless/uid','rb') as f:
    uid = pickle.load(f)

In [12]:

import pickle
with open('../../../IMDB_250_Backup/Useless/reviews','rb') as f:
    reviews = pickle.load(f)
 

In [5]:
df = pd.read_csv('../../../IMDB_250_Backup/top_movies.csv')
id_list = list(df.ID)

In [2]:
pd.read_csv('top_movies.csv')

,Unnamed: 0,ID,Rate,Title,Vote,Year
0,0,tt0111161,9.3,The Shawshank Redemption,1718456,1994
1,1,tt0068646,9.2,The Godfather,1174559,1972
2,2,tt0071562,9.0,The Godfather: Part II,804922,1974
3,3,tt0468569,9.0,The Dark Knight,1704632,2008
4,4,tt0050083,8.9,12 Angry Men,457572,1957
5,5,tt0108052,8.9,Schindler's List,880006,1993
6,6,tt0110912,8.9,Pulp Fiction,1346366,1994
7,7,tt0167260,8.9,The Lord of the Rings: The Return of the King,1234876,2003
8,8,tt0060196,8.9,"The Good, the Bad and the Ugly",511565,1966
9,9,tt0137523,8.8,Fight Club,1371582,1999


In [6]:
# To build star dataframe
total = []
for i in id_list:
    df = pd.DataFrame({i: star[id_list.index(i)]})
    total.append(df)
star = pd.concat(total, axis=1)

In [8]:
# To build uid dataframe
total = []
for i in id_list:
    df = pd.DataFrame({i: uid[id_list.index(i)]})
    total.append(df)
uid = pd.concat(total, axis=1)

In [9]:
def fff(x):
    try:
        if np.isnan(x):
            return
    except:
        return x[6:-1]
uid = uid.applymap(fff)

In [13]:
# To build review dataframe
total = []
for i in id_list:
    df = pd.DataFrame({i: reviews[id_list.index(i)]})
    total.append(df)
reviews = pd.concat(total, axis=1)

In [15]:
uid.to_csv('./uid.csv')
star.to_csv('./star.csv')
reviews.to_csv('./reviews.csv', encoding='utf-8')

## To Create dataframe with UID MovieID Rating columns

In [16]:
top = pd.read_csv('top_movies.csv')
id_list = list(top.ID)

uid = pd.read_csv('uid.csv')
star = pd.read_csv('star.csv')
uid = uid.iloc[:,1:]

"""
reviews = pd.read_csv('reviews.csv')
reviews = reviews.iloc[:,1:]
"""
#Convert 10/10, 8/10 .... to 10 and 8...
def fff(x):
    try:
        if x.endswith('/10'):
            return int(x[:x.find('/')])
    except:
        return
star = star.iloc[:,1:].applymap(fff)

//anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (2,3,5,6,7,8,9,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250) have mixed ty

In [18]:
# To create MID_UID dataframe
complex_table = pd.DataFrame(columns = ['MID','UID'])
tables = []
for i in id_list:
    table = pd.melt(uid,value_vars=[i],var_name = 'MID', value_name = 'UID')
    tables.append(table)
MID_UID = complex_table.append(tables,ignore_index=True)

In [21]:
# To create MID_STAR dataframe
complex_table = pd.DataFrame(columns = ['MID','STAR'])
tables = []
for i in id_list:
    table = pd.melt(star,value_vars=[i],var_name = 'MID', value_name = 'STAR')
    tables.append(table)
MID_STAR = complex_table.append(tables,ignore_index=True)

In [23]:
# To create MID_STAR_UID dataframe
result = pd.concat([MID_STAR,MID_UID],axis=1,ignore_index=True)
result.columns = ['MID','STAR','MID2','UID']
result.drop('MID2',axis=1,inplace=True)
result

,MID,STAR,UID
0,tt0111161,10.0,ur2509775
1,tt0111161,10.0,ur0997166
2,tt0111161,NaN,ur1005460
3,tt0111161,10.0,ur1898687
4,tt0111161,NaN,ur0094239
5,tt0111161,8.0,ur1285640
6,tt0111161,10.0,ur0611718
7,tt0111161,10.0,ur0257957
8,tt0111161,10.0,ur0355122
9,tt0111161,10.0,ur0842118


In [25]:
# To delete NAN in UID column
result = result[result['UID'].isnull() == False]
result

,MID,STAR,UID
0,tt0111161,10.0,ur2509775
1,tt0111161,10.0,ur0997166
2,tt0111161,NaN,ur1005460
3,tt0111161,10.0,ur1898687
4,tt0111161,NaN,ur0094239
5,tt0111161,8.0,ur1285640
6,tt0111161,10.0,ur0611718
7,tt0111161,10.0,ur0257957
8,tt0111161,10.0,ur0355122
9,tt0111161,10.0,ur0842118


In [26]:
# Fill NAN with mean in each group

MID_STAR_UID = result.groupby('MID').transform(lambda x: x.fillna(x.mean()))

In [27]:
# Add MID column back and change 'STAR' 'UID' type
MID_STAR_UID = pd.concat([MID_STAR_UID,result[['MID']]], axis = 1)
MID_STAR_UID['STAR'] = MID_STAR_UID['STAR'].astype(int)
#MID_STAR_UID['UID'] = MID_STAR_UID['UID'].astype(int)
MID_STAR_UID

,STAR,MID
0,10,tt0111161
1,10,tt0111161
2,9,tt0111161
3,10,tt0111161
4,9,tt0111161
5,8,tt0111161
6,10,tt0111161
7,10,tt0111161
8,10,tt0111161
9,10,tt0111161


In [32]:
MID_STAR_UID = pd.concat([MID_STAR_UID,result[["UID"]]], axis = 1)
MID_STAR_UID

,STAR,UID,MID
0,10,ur2509775,tt0111161
1,10,ur0997166,tt0111161
2,9,ur1005460,tt0111161
3,10,ur1898687,tt0111161
4,9,ur0094239,tt0111161
5,8,ur1285640,tt0111161
6,10,ur0611718,tt0111161
7,10,ur0257957,tt0111161
8,10,ur0355122,tt0111161
9,10,ur0842118,tt0111161


In [34]:
pivot = MID_STAR_UID.pivot_table(values='STAR',index='UID',columns='MID')
pivot

MID,tt0012349,tt0015864,tt0017136,tt0017925,tt0018455,tt0019254,tt0021749,tt0022100,tt0025316,tt0027977,...,tt2096673,tt2106476,tt2267998,tt2278388,tt2488496,tt2582802,tt2948356,tt3011894,tt3170832,tt3659388
UID,,,,,,,,,,,,,,,,,,,,,
ur0000006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ur0000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ur0000037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ur0000039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ur0000059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ur0000066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ur0000075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ur0000138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ur0000157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
print "Number of reviews: " + str(len(MID_STAR_UID))
print "Number of users: " + str(pivot.index.shape[0])
print "Number of movies: " + str(len(pivot.columns))

Number of reviews: 205708
Number of users: 111006
Number of movies: 250


In [38]:
MID_STAR_UID.to_csv('MID_STAR_UID.csv')

In [39]:
pivot.to_csv('pivot.csv')

### Dataset Extension

In [268]:
import pickle
with open('../../../Extra_movies/uid','rb') as f:
    uid_ext = pickle.load(f)

In [277]:
uid_ext = uid_ext.T.groupby(level=0).first().T

In [290]:
import pickle
with open('../../../Extra_movies/star','rb') as f:
    star_ext = pickle.load(f)

In [293]:
star_ext = star_ext.T.groupby(level=0).first().T

In [297]:
star_ext.to_csv('star_ext.csv')

In [304]:
import pickle
with open('../../../Extra_movies/reviews','rb') as f:
    reviews_ext = pickle.load(f)

In [305]:
reviews_ext =reviews_ext.T.groupby(level=0).first().T

In [307]:
reviews_ext.to_csv('reviews_ext.csv',encoding='utf-8')

In [333]:
def fff(x):
    if np.isnan(x) == False:
        return 'ur'+ str(int(x))
uid_ext = uid_ext.applymap(fff)

In [335]:
uid_ext.to_csv('uid_ext.csv')

In [336]:
uid_ext = pd.read_csv('uid_ext.csv')

In [338]:
del uid_ext['Unnamed: 0']

In [344]:
id_list_ext = uid_ext.columns.values


In [345]:
# To create MID_UID dataframe
complex_table = pd.DataFrame(columns = ['MID','UID'])
tables = []
for i in id_list_ext:
    table = pd.melt(uid_ext,value_vars=[i],var_name = 'MID', value_name = 'UID')
    tables.append(table)
MID_UID_EXT = complex_table.append(tables,ignore_index=True)

In [348]:
star_ext = pd.read_csv('star_ext.csv')

In [349]:
del star_ext['Unnamed: 0']

In [351]:
# To create MID_STAR dataframe
complex_table = pd.DataFrame(columns = ['MID','STAR'])
tables = []
for i in id_list_ext:
    table = pd.melt(star_ext,value_vars=[i],var_name = 'MID', value_name = 'STAR')
    tables.append(table)
MID_STAR_EXT = complex_table.append(tables,ignore_index=True)

In [353]:
# To create MID_STAR_UID dataframe
result = pd.concat([MID_STAR_EXT,MID_UID_EXT],axis=1,ignore_index=True)
result.columns = ['MID','STAR','MID2','UID']
result.drop('MID2',axis=1,inplace=True)
result

,MID,STAR,UID
0,tt0070047,10.0,ur2286500
1,tt0070047,NaN,ur1298472
2,tt0070047,10.0,ur176175
3,tt0070047,10.0,ur660441
4,tt0070047,10.0,ur1293485
5,tt0070047,10.0,ur4216884
6,tt0070047,NaN,ur1377414
7,tt0070047,10.0,ur6880633
8,tt0070047,9.0,ur19658928
9,tt0070047,10.0,ur375889


In [354]:
# To delete NAN in UID column
result = result[result['UID'].isnull() == False]
result

,MID,STAR,UID
0,tt0070047,10.0,ur2286500
1,tt0070047,NaN,ur1298472
2,tt0070047,10.0,ur176175
3,tt0070047,10.0,ur660441
4,tt0070047,10.0,ur1293485
5,tt0070047,10.0,ur4216884
6,tt0070047,NaN,ur1377414
7,tt0070047,10.0,ur6880633
8,tt0070047,9.0,ur19658928
9,tt0070047,10.0,ur375889


In [357]:
# Fill NAN with mean in each group

MID_STAR_UID_EXT = result.fillna(result.groupby('MID').transform('mean'))

In [367]:
MID_STAR_UID_EXT['STAR'] = MID_STAR_UID_EXT['STAR'].astype(int)

In [369]:
MID_STAR_UID_EXT.to_csv('MID_STAR_UID_EXT.csv')

In [2]:
MID_STAR_UID_EXT = pd.read_csv('MID_STAR_UID_EXT.csv')

In [4]:
del MID_STAR_UID_EXT['Unnamed: 0']

In [9]:
cols = MID_STAR_UID_EXT.columns.tolist()

In [11]:
cols = ['MID','UID','STAR']

In [12]:
MID_STAR_UID_EXT = MID_STAR_UID_EXT[cols]

In [14]:
MID_STAR_UID = pd.read_csv('MID_STAR_UID.csv')

In [15]:
del MID_STAR_UID['Unnamed: 0']

In [17]:
cols = MID_STAR_UID.columns.tolist()

In [18]:
cols = ['MID','UID','STAR']

In [19]:
MID_STAR_UID = MID_STAR_UID[cols]

In [21]:
###### concat MID_STAR_UID and MID_STAR_UID_EXT ########

In [25]:
MID_UID_STAR_COM = pd.concat([MID_STAR_UID,MID_STAR_UID_EXT],axis=0,ignore_index=True)

In [27]:
MID_UID_STAR_COM.to_csv('MID_UID_STAR_COM.csv')

In [28]:
MID_UID_STAR_COM = pd.read_csv('MID_UID_STAR_COM.csv')

In [30]:
del MID_UID_STAR_COM['Unnamed: 0']

In [32]:
pivot_com = MID_UID_STAR_COM.pivot_table(values='STAR',index='UID',columns='MID')
pivot_com

MID,tt0012349,tt0015864,tt0017136,tt0017925,tt0018455,tt0019254,tt0021749,tt0022100,tt0025316,tt0027977,...,tt2582802,tt2788710,tt2872732,tt2948356,tt2975590,tt3011894,tt3170832,tt3460252,tt3498820,tt3659388
UID,,,,,,,,,,,,,,,,,,,,,
ur0000006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ur0000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ur0000037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ur0000039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ur0000059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ur0000066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ur0000075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ur0000138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ur0000157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
print "Number of reviews: " + str(len(MID_UID_STAR_COM))
print "Number of users: " + str(pivot_com.index.shape[0])
print "Number of movies: " + str(len(pivot_com.columns))

Number of reviews: 528879
Number of users: 254745
Number of movies: 481


In [34]:
pivot_com.to_csv('pivot_com.csv')


## Cosine similarity

In [3]:
df = pd.read_csv('pivot_com.csv',index_col='UID')
df.fillna(0,inplace=True)
df

,tt0012349,tt0015864,tt0017136,tt0017925,tt0018455,tt0019254,tt0021749,tt0022100,tt0025316,tt0027977,...,tt2582802,tt2788710,tt2872732,tt2948356,tt2975590,tt3011894,tt3170832,tt3460252,tt3498820,tt3659388
UID,,,,,,,,,,,,,,,,,,,,,
ur0000006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ur0000011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ur0000037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ur0000039,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ur0000059,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ur0000066,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ur0000075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ur0000138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ur0000157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df_bigger_100 = df[df.sum(axis=1)>100]
df_bigger_100

,tt0012349,tt0015864,tt0017136,tt0017925,tt0018455,tt0019254,tt0021749,tt0022100,tt0025316,tt0027977,...,tt2582802,tt2788710,tt2872732,tt2948356,tt2975590,tt3011894,tt3170832,tt3460252,tt3498820,tt3659388
UID,,,,,,,,,,,,,,,,,,,,,
ur0002746,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ur0003136,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ur0011596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ur0011762,0.0,0.0,0.0,7.0,0.0,7.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0
ur0018365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ur0020866,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ur0023796,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ur0024463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ur0033913,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df_bigger_100 = df_bigger_100.unstack().reset_index(name='value')

In [6]:
# all stars including 0 from 2742 users
df_bigger_100

,level_0,UID,value
0,tt0012349,ur0002746,0.0
1,tt0012349,ur0003136,0.0
2,tt0012349,ur0011596,0.0
3,tt0012349,ur0011762,0.0
4,tt0012349,ur0018365,0.0
5,tt0012349,ur0020866,0.0
6,tt0012349,ur0023796,0.0
7,tt0012349,ur0024463,0.0
8,tt0012349,ur0033913,0.0
9,tt0012349,ur0035229,0.0


In [7]:
# delete all 0 stars
df = df_bigger_100[df_bigger_100['value'] != 0]

In [8]:
df = df.reset_index(drop=True)

In [9]:
df.columns = ['MID','UID','value']

In [10]:
print "Number of reviews: " + str(len(df))
print "Number of users: " + str(df['UID'].unique().shape[0])
print "Number of movies: " + str(df['MID'].unique().shape[0])

Number of reviews: 104968
Number of users: 2742
Number of movies: 481


In [11]:
# Movie Menu

movie_menu = pd.DataFrame({'MID': df['MID'].unique().tolist(),
                          'Code': np.arange(1,482)})

In [12]:
# User menu

In [13]:
uid_unique = df['UID'].unique().tolist()
uid_unique_sort = sorted(uid_unique)

user_menu = pd.DataFrame({'UID': uid_unique_sort,
                          'Code': np.arange(1,2743)})

In [118]:
# To create sequence table
user_code = []
for i in df['UID']:
    user_code.append(user_menu['UID'].tolist().index(i)+1)

movie_code = []
for i in df['MID']:
    movie_code.append(movie_menu['MID'].tolist().index(i)+1)

seq_100 = pd.DataFrame({'MID': movie_code,
                        "UID": user_code,
                        'STAR': df['value']
                       })

In [ ]:
#### More features

In [ ]:
mega = pd.read_csv('movie_metadata.csv')

check = []
for i in mega.movie_imdb_link:
    check.append(i[26:(i.find('?')-1)])

common = []
for i in mid_list:
    if i in check:
        common.append(i)

def fff(x):
    return x[26:35]
mega.movie_imdb_link = mega.movie_imdb_link.apply(fff)

mega.drop_duplicates(subset = 'movie_imdb_link', inplace = True)

mega.set_index('movie_imdb_link',inplace=True)



lst=[]
for i in common:
    lst.append(mega.loc[[i]])

df_feature = pd.concat(lst)

In [319]:
df_feature.shape

(401, 27)

In [323]:
df_feature

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
movie_imdb_link,,,,,,,,,,,,,,,,,,,,,
tt0017136,Black and White,Fritz Lang,260.0,145.0,756.0,18.0,Gustav Fröhlich,136.0,26435.0,Drama|Sci-Fi,...,413.0,German,Germany,Not Rated,6000000.0,1927.0,23.0,8.3,1.33,12000
tt0025316,Black and White,Frank Capra,124.0,65.0,964.0,21.0,Alan Hale,380.0,NaN,Comedy|Romance,...,235.0,English,USA,Unrated,325000.0,1934.0,114.0,8.2,1.37,0
tt0027977,Black and White,Charles Chaplin,120.0,87.0,0.0,8.0,Stanley Blystone,309.0,163245.0,Comedy|Drama|Family,...,211.0,English,USA,G,1500000.0,1936.0,8.0,8.6,1.37,0
tt0031381,Color,Victor Fleming,157.0,226.0,149.0,248.0,George Reeves,503.0,198655278.0,Drama|History|Romance|War,...,706.0,English,USA,G,3977000.0,1939.0,384.0,8.2,1.37,16000
tt0031679,Black and White,Frank Capra,96.0,120.0,964.0,248.0,Jean Arthur,607.0,NaN,Comedy|Drama,...,245.0,English,USA,Not Rated,1500000.0,1939.0,319.0,8.2,1.37,0
tt0032138,Black and White,Victor Fleming,213.0,102.0,149.0,357.0,Terry,695.0,22202612.0,Adventure|Family|Fantasy|Musical,...,533.0,English,USA,Passed,2800000.0,1939.0,421.0,8.1,1.37,14000
tt0032976,Black and White,Alfred Hitchcock,144.0,130.0,13000.0,333.0,Joan Fontaine,1000.0,NaN,Drama|Film-Noir|Mystery|Thriller,...,276.0,English,USA,Not Rated,1288000.0,1940.0,991.0,8.2,1.37,0
tt0034583,Color,Michael Curtiz,242.0,82.0,345.0,269.0,Claude Rains,2000.0,NaN,Drama|Romance|War,...,1123.0,English,USA,PG,950000.0,1942.0,607.0,8.6,1.37,23000
tt0036868,Black and White,William Wyler,97.0,172.0,355.0,188.0,Teresa Wright,749.0,23650000.0,Drama|Romance|War,...,235.0,English,USA,Not Rated,2100000.0,1946.0,208.0,8.1,1.37,0


In [330]:
df_feature.to_csv('features.csv')

In [ ]:
#From current_dataset.csv to final.csv
df = pd.read_csv('current_dataset.csv')

del df['Unnamed: 0']

In [ ]:
print "Number of reviews: " + str(len(df))
print "Number of users: " + str(df['UID'].unique().shape[0])
print "Number of movies: " + str(df['MID'].unique().shape[0])

"""
Number of reviews: 104968
Number of users: 2742
Number of movies: 481
"""

In [ ]:
features = pd.read_csv('features.csv')

movie_data = features[['movie_imdb_link','movie_title','genres']]

In [ ]:
movie_data_id = movie_data['movie_imdb_link'].tolist()

In [ ]:
df = df[df['MID'].isin(movie_data_id)]

df.reset_index(inplace=True)

del df['index']

In [ ]:
print "Number of reviews: " + str(len(df))
print "Number of users: " + str(df['UID'].unique().shape[0])
print "Number of movies: " + str(df['MID'].unique().shape[0])

"""
Number of reviews: 95233
Number of users: 2742
Number of movies: 401
"""